## 浙江省财政支出规模与增长匹配度

In [22]:
import polars as pl
import altair as alt
import pandas as pd
import numpy as np

# ==============================================
# 1. 读取GDP数据
# ==============================================
gdp_df = pl.read_excel(
    "../data/1-5 全省生产总值（1978-2023年）.xlsx",
    sheet_name="1-5 全省生产总值（1978-2023年）",
    read_options={"header_row": 1}  # 作为列名
)

print("GDP数据列名：", gdp_df.columns)
print("\nGDP数据预览（2018-2023）：")

# 重命名列
gdp_df = gdp_df.rename({
    gdp_df.columns[0]: "年份",
    gdp_df.columns[1]: "全省生产总值(亿元)"
})

# 筛选2018-2023年的数据
gdp_recent = gdp_df.filter(
    (pl.col("年份") >= "2018") & (pl.col("年份") <= "2023")
).select(["年份", "全省生产总值(亿元)"])

print(gdp_recent)


GDP数据列名： ['年份', '全省生产总值\n(亿元)', '第一产业\n', '第二产业\n', '第三产业\n', '工业\n', '人均生产\n总值(元)']

GDP数据预览（2018-2023）：
shape: (6, 2)
┌──────┬────────────────────┐
│ 年份 ┆ 全省生产总值(亿元) │
│ ---  ┆ ---                │
│ str  ┆ f64                │
╞══════╪════════════════════╡
│ 2018 ┆ 58002.84           │
│ 2019 ┆ 62462.0            │
│ 2020 ┆ 64689.06           │
│ 2021 ┆ 74040.78           │
│ 2022 ┆ 78060.63           │
│ 2023 ┆ 82553.21           │
└──────┴────────────────────┘


In [23]:
# ==============================================
# 2. 读取财政收支数据
# ==============================================
fiscal_df = pl.read_excel(
    "../data/12-2 地方一般预算收入和支出（2018-2022年）.xlsx",
    sheet_name="12-2 地方一般预算收入和支出（2018-2022年）",
    read_options={"header_row": 2}
)

# 提取收入和支出数据
income_row = fiscal_df.filter(pl.col("项目") == "一般预算收入合计")
expense_row = fiscal_df.filter(pl.col("项目") == "一般预算支出合计")

years = ["2018", "2019", "2020", "2021", "2022", "2023"]

# print(fiscal_df)
print("\n财政收入数据预览：", income_row)
print("\n财政支出数据预览：", expense_row)


财政收入数据预览： shape: (1, 7)
┌──────────────────┬─────────┬────────┬─────────┬─────────┬─────────┬─────────┐
│ 项目             ┆ 2018    ┆ 2019   ┆ 2020    ┆ 2021    ┆ 2022    ┆ 2023    │
│ ---              ┆ ---     ┆ ---    ┆ ---     ┆ ---     ┆ ---     ┆ ---     │
│ str              ┆ f64     ┆ f64    ┆ f64     ┆ f64     ┆ f64     ┆ f64     │
╞══════════════════╪═════════╪════════╪═════════╪═════════╪═════════╪═════════╡
│ 一般预算收入合计 ┆ 6598.08 ┆ 7048.0 ┆ 7248.24 ┆ 8262.57 ┆ 8039.38 ┆ 8600.02 │
└──────────────────┴─────────┴────────┴─────────┴─────────┴─────────┴─────────┘

财政支出数据预览： shape: (1, 7)
┌──────────────────┬─────────┬──────────┬──────────┬──────────┬─────────┬──────────┐
│ 项目             ┆ 2018    ┆ 2019     ┆ 2020     ┆ 2021     ┆ 2022    ┆ 2023     │
│ ---              ┆ ---     ┆ ---      ┆ ---      ┆ ---      ┆ ---     ┆ ---      │
│ str              ┆ f64     ┆ f64      ┆ f64      ┆ f64      ┆ f64     ┆ f64      │
╞══════════════════╪═════════╪══════════╪══════════╪══════════

In [24]:
# ==============================================
# 3. 创建综合分析数据表
# ==============================================
analysis_data = {
    "年份": years,
    "一般预算收入合计": [float(income_row[year][0]) for year in years],
    "一般预算支出合计": [float(expense_row[year][0]) for year in years]
}
print("\n综合分析数据预览：", analysis_data)
df_analysis = pl.DataFrame(analysis_data)
print(df_analysis)


综合分析数据预览： {'年份': ['2018', '2019', '2020', '2021', '2022', '2023'], '一般预算收入合计': [6598.08, 7048.0, 7248.24, 8262.57, 8039.38, 8600.02], '一般预算支出合计': [8627.51, 10053.03, 10082.01, 11016.87, 12017.7, 12353.09]}
shape: (6, 3)
┌──────┬──────────────────┬──────────────────┐
│ 年份 ┆ 一般预算收入合计 ┆ 一般预算支出合计 │
│ ---  ┆ ---              ┆ ---              │
│ str  ┆ f64              ┆ f64              │
╞══════╪══════════════════╪══════════════════╡
│ 2018 ┆ 6598.08          ┆ 8627.51          │
│ 2019 ┆ 7048.0           ┆ 10053.03         │
│ 2020 ┆ 7248.24          ┆ 10082.01         │
│ 2021 ┆ 8262.57          ┆ 11016.87         │
│ 2022 ┆ 8039.38          ┆ 12017.7          │
│ 2023 ┆ 8600.02          ┆ 12353.09         │
└──────┴──────────────────┴──────────────────┘


In [25]:
# 合并GDP数据
df_analysis = df_analysis.join(
    gdp_recent.rename({"年份": "年份"}),
    on="年份"
)
print("\n合并GDP后的综合分析数据预览：", df_analysis)


合并GDP后的综合分析数据预览： shape: (6, 4)
┌──────┬──────────────────┬──────────────────┬────────────────────┐
│ 年份 ┆ 一般预算收入合计 ┆ 一般预算支出合计 ┆ 全省生产总值(亿元) │
│ ---  ┆ ---              ┆ ---              ┆ ---                │
│ str  ┆ f64              ┆ f64              ┆ f64                │
╞══════╪══════════════════╪══════════════════╪════════════════════╡
│ 2018 ┆ 6598.08          ┆ 8627.51          ┆ 58002.84           │
│ 2019 ┆ 7048.0           ┆ 10053.03         ┆ 62462.0            │
│ 2020 ┆ 7248.24          ┆ 10082.01         ┆ 64689.06           │
│ 2021 ┆ 8262.57          ┆ 11016.87         ┆ 74040.78           │
│ 2022 ┆ 8039.38          ┆ 12017.7          ┆ 78060.63           │
│ 2023 ┆ 8600.02          ┆ 12353.09         ┆ 82553.21           │
└──────┴──────────────────┴──────────────────┴────────────────────┘


In [27]:
# 计算财政收入/GDP占比
df_analysis = df_analysis.with_columns([
    (pl.col("一般预算收入合计") / pl.col("全省生产总值(亿元)") * 100)
    .alias("财政收入/GDP占比(%)")
])
print("\n添加财政收入/GDP占比后的数据预览：", df_analysis)


添加财政收入/GDP占比后的数据预览： shape: (6, 5)
┌──────┬──────────────────┬──────────────────┬────────────────────┬─────────────────────┐
│ 年份 ┆ 一般预算收入合计 ┆ 一般预算支出合计 ┆ 全省生产总值(亿元) ┆ 财政收入/GDP占比(%) │
│ ---  ┆ ---              ┆ ---              ┆ ---                ┆ ---                 │
│ str  ┆ f64              ┆ f64              ┆ f64                ┆ f64                 │
╞══════╪══════════════════╪══════════════════╪════════════════════╪═════════════════════╡
│ 2018 ┆ 6598.08          ┆ 8627.51          ┆ 58002.84           ┆ 11.375443           │
│ 2019 ┆ 7048.0           ┆ 10053.03         ┆ 62462.0            ┆ 11.28366            │
│ 2020 ┆ 7248.24          ┆ 10082.01         ┆ 64689.06           ┆ 11.204738           │
│ 2021 ┆ 8262.57          ┆ 11016.87         ┆ 74040.78           ┆ 11.159485           │
│ 2022 ┆ 8039.38          ┆ 12017.7          ┆ 78060.63           ┆ 10.298892           │
│ 2023 ┆ 8600.02          ┆ 12353.09         ┆ 82553.21           ┆ 10.417548           │
└──────

In [29]:
# 计算收支增长率
df_analysis = df_analysis.with_columns([
    (pl.col("一般预算收入合计").pct_change() * 100)
    .alias("收入同比增长率(%)"),
    (pl.col("一般预算支出合计").pct_change() * 100)
    .alias("支出同比增长率(%)"),
    (pl.col("全省生产总值(亿元)").pct_change() * 100)
    .alias("GDP同比增长率(%)")
])
print("\n添加同比增长率后的数据预览：", df_analysis)


添加同比增长率后的数据预览： shape: (6, 8)
┌──────┬─────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┐
│ 年份 ┆ 一般预算收  ┆ 一般预算支 ┆ 全省生产总 ┆ 财政收入/G ┆ 收入同比增 ┆ 支出同比增 ┆ GDP同比增  │
│ ---  ┆ 入合计      ┆ 出合计     ┆ 值(亿元)   ┆ DP占比(%)  ┆ 长率(%)    ┆ 长率(%)    ┆ 长率(%)    │
│ str  ┆ ---         ┆ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│      ┆ f64         ┆ f64        ┆ f64        ┆ f64        ┆ f64        ┆ f64        ┆ f64        │
╞══════╪═════════════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╡
│ 2018 ┆ 6598.08     ┆ 8627.51    ┆ 58002.84   ┆ 11.375443  ┆ null       ┆ null       ┆ null       │
│ 2019 ┆ 7048.0      ┆ 10053.03   ┆ 62462.0    ┆ 11.28366   ┆ 6.818953   ┆ 16.52296   ┆ 7.68783    │
│ 2020 ┆ 7248.24     ┆ 10082.01   ┆ 64689.06   ┆ 11.204738  ┆ 2.84109    ┆ 0.288271   ┆ 3.565464   │
│ 2021 ┆ 8262.57     ┆ 11016.87   ┆ 74040.78   ┆ 11.159485  ┆ 13.994156  ┆ 9.272556   ┆ 14.456417  │
│ 2022 ┆ 80

In [31]:
# 计算累计增长率
base_2018_income = df_analysis.filter(pl.col("年份") == "2018")["一般预算收入合计"][0]
base_2018_expense = df_analysis.filter(pl.col("年份") == "2018")["一般预算支出合计"][0]
base_2018_gdp = df_analysis.filter(pl.col("年份") == "2018")["全省生产总值(亿元)"][0]
df_analysis = df_analysis.with_columns([
    ((pl.col("一般预算收入合计") - base_2018_income) / base_2018_income * 100)
    .alias("收入累计增长率(%)"),
    ((pl.col("一般预算支出合计") - base_2018_expense) / base_2018_expense * 100)
    .alias("支出累计增长率(%)"),
    ((pl.col("全省生产总值(亿元)") - base_2018_gdp) / base_2018_gdp * 100)
    .alias("GDP累计增长率(%)")
])
print("\n添加累计增长率后的数据预览：", df_analysis)


添加累计增长率后的数据预览： shape: (6, 11)
┌──────┬────────────┬────────────┬────────────┬───┬────────────┬───────────┬───────────┬───────────┐
│ 年份 ┆ 一般预算收 ┆ 一般预算支 ┆ 全省生产总 ┆ … ┆ GDP同比增  ┆ 收入累计  ┆ 支出累计  ┆ GDP累计增 │
│ ---  ┆ 入合计     ┆ 出合计     ┆ 值(亿元)   ┆   ┆ 长率(%)    ┆ 增长率(%) ┆ 增长率(%) ┆ 长率(%)   │
│ str  ┆ ---        ┆ ---        ┆ ---        ┆   ┆ ---        ┆ ---       ┆ ---       ┆ ---       │
│      ┆ f64        ┆ f64        ┆ f64        ┆   ┆ f64        ┆ f64       ┆ f64       ┆ f64       │
╞══════╪════════════╪════════════╪════════════╪═══╪════════════╪═══════════╪═══════════╪═══════════╡
│ 2018 ┆ 6598.08    ┆ 8627.51    ┆ 58002.84   ┆ … ┆ null       ┆ 0.0       ┆ 0.0       ┆ 0.0       │
│ 2019 ┆ 7048.0     ┆ 10053.03   ┆ 62462.0    ┆ … ┆ 7.68783    ┆ 6.818953  ┆ 16.52296  ┆ 7.68783   │
│ 2020 ┆ 7248.24    ┆ 10082.01   ┆ 64689.06   ┆ … ┆ 3.565464   ┆ 9.853776  ┆ 16.858862 ┆ 11.527401 │
│ 2021 ┆ 8262.57    ┆ 11016.87   ┆ 74040.78   ┆ … ┆ 14.456417  ┆ 25.226884 ┆ 27.694665 ┆ 27.650267 │
│ 2022 ┆ 

In [32]:

# 计算财政赤字和收支比
df_analysis = df_analysis.with_columns([
    (pl.col("一般预算支出合计") - pl.col("一般预算收入合计")).alias("财政赤字(亿元)"),
    (pl.col("一般预算支出合计") / pl.col("一般预算收入合计") * 100).alias("支出收入比(%)")
])
print("\n添加财政赤字和收支比后的数据预览：", df_analysis)


添加财政赤字和收支比后的数据预览： shape: (6, 13)
┌──────┬────────────┬────────────┬────────────┬───┬────────────┬───────────┬───────────┬───────────┐
│ 年份 ┆ 一般预算收 ┆ 一般预算支 ┆ 全省生产总 ┆ … ┆ 支出累计增 ┆ GDP累计增 ┆ 财政赤字( ┆ 支出收入  │
│ ---  ┆ 入合计     ┆ 出合计     ┆ 值(亿元)   ┆   ┆ 长率(%)    ┆ 长率(%)   ┆ 亿元)     ┆ 比(%)     │
│ str  ┆ ---        ┆ ---        ┆ ---        ┆   ┆ ---        ┆ ---       ┆ ---       ┆ ---       │
│      ┆ f64        ┆ f64        ┆ f64        ┆   ┆ f64        ┆ f64       ┆ f64       ┆ f64       │
╞══════╪════════════╪════════════╪════════════╪═══╪════════════╪═══════════╪═══════════╪═══════════╡
│ 2018 ┆ 6598.08    ┆ 8627.51    ┆ 58002.84   ┆ … ┆ 0.0        ┆ 0.0       ┆ 2029.43   ┆ 130.75788 │
│      ┆            ┆            ┆            ┆   ┆            ┆           ┆           ┆ 7         │
│ 2019 ┆ 7048.0     ┆ 10053.03   ┆ 62462.0    ┆ … ┆ 16.52296   ┆ 7.68783   ┆ 3005.03   ┆ 142.63663 │
│      ┆            ┆            ┆            ┆   ┆            ┆           ┆           ┆ 5         │
│ 202

In [33]:
# ==============================================
# 4. 计算关键指标
# ==============================================
years_count = 5
cagr_income = ((df_analysis["一般预算收入合计"][-1] / df_analysis["一般预算收入合计"][0]) ** (1/years_count) - 1) * 100
cagr_expense = ((df_analysis["一般预算支出合计"][-1] / df_analysis["一般预算支出合计"][0]) ** (1/years_count) - 1) * 100
cagr_gdp = ((df_analysis["全省生产总值(亿元)"][-1] / df_analysis["全省生产总值(亿元)"][0]) ** (1/years_count) - 1) * 100

print("=" * 70)
print("浙江省财政支出规模与增长匹配度分析（2018-2023年）")
print("=" * 70)

print("\n一、基础数据：")
print(df_analysis.select([
    "年份", 
    "全省生产总值(亿元)",
    "一般预算收入合计",
    "一般预算支出合计",
    "财政收入/GDP占比(%)"
]).to_pandas().to_string(index=False))

print("\n二、增长情况：")
print(df_analysis.select([
    "年份", 
    "收入同比增长率(%)",
    "支出同比增长率(%)",
    "GDP同比增长率(%)",
    "收入累计增长率(%)",
    "支出累计增长率(%)",
    "GDP累计增长率(%)"
]).to_pandas().to_string(index=False))

print(f"\n三、关键指标分析：")
print(f"1. 支出累计增长: {df_analysis['支出累计增长率(%)'][-1]:.2f}%")
print(f"2. 收入累计增长: {df_analysis['收入累计增长率(%)'][-1]:.2f}%")
print(f"3. GDP累计增长: {df_analysis['GDP累计增长率(%)'][-1]:.2f}%")
print(f"4. 支出年均复合增长率(CAGR): {cagr_expense:.2f}%")
print(f"5. 收入年均复合增长率(CAGR): {cagr_income:.2f}%")
print(f"6. GDP年均复合增长率(CAGR): {cagr_gdp:.2f}%")
print(f"7. 平均财政收入/GDP占比: {df_analysis['财政收入/GDP占比(%)'].mean():.2f}%")
print(f"8. 平均财政赤字: {df_analysis['财政赤字(亿元)'].mean():.2f}亿元")
print(f"9. 平均支出收入比: {df_analysis['支出收入比(%)'].mean():.2f}%")

浙江省财政支出规模与增长匹配度分析（2018-2023年）

一、基础数据：
  年份  全省生产总值(亿元)  一般预算收入合计  一般预算支出合计  财政收入/GDP占比(%)
2018    58002.84   6598.08   8627.51      11.375443
2019    62462.00   7048.00  10053.03      11.283660
2020    64689.06   7248.24  10082.01      11.204738
2021    74040.78   8262.57  11016.87      11.159485
2022    78060.63   8039.38  12017.70      10.298892
2023    82553.21   8600.02  12353.09      10.417548

二、增长情况：
  年份  收入同比增长率(%)  支出同比增长率(%)  GDP同比增长率(%)  收入累计增长率(%)  支出累计增长率(%)  GDP累计增长率(%)
2018         NaN         NaN          NaN    0.000000    0.000000     0.000000
2019    6.818953   16.522960     7.687830    6.818953   16.522960     7.687830
2020    2.841090    0.288271     3.565464    9.853776   16.858862    11.527401
2021   13.994156    9.272556    14.456417   25.226884   27.694665    27.650267
2022   -2.701218    9.084522     5.429238   21.844233   39.295115    34.580703
2023    6.973672    2.790800     5.755244   30.341251   43.182564    42.326152

三、关键指标分析：
1. 支出累计增长: 43.18%
2. 收入累

In [35]:
# ==============================================
# 5. 数据可视化
# ==============================================
df_pd = df_analysis.to_pandas()

# 1. 财政支出规模与增速对比图
chart1 = alt.Chart(df_pd).mark_bar(
    width=35,
    opacity=0.8
).encode(
    x=alt.X('年份:N', axis=alt.Axis(title='年份', labelAngle=0)),
    y=alt.Y('一般预算支出合计:Q', 
           axis=alt.Axis(title='一般预算支出（亿元）', titleColor='#F58518')),
    color=alt.value('#F58518'),
    tooltip=['年份', alt.Tooltip('一般预算支出合计:Q', format=',.0f')]
)

# 添加支出增长率线
line_expense = alt.Chart(df_pd.iloc[1:]).mark_line(
    color='#E45756',
    strokeWidth=3
).encode(
    x='年份:N',
    y=alt.Y('支出同比增长率(%):Q', 
           axis=alt.Axis(title='支出同比增长率(%)', titleColor='#E45756'))
)

chart1_combo = alt.layer(chart1, line_expense).resolve_scale(
    y='independent'
).properties(
    title='浙江省一般预算支出规模与增速（2018-2023年）',
    width=600,
    height=400
)
chart1_combo.show()

alt.LayerChart(...)

In [36]:
# 2. 收入、支出、GDP增速对比图
df_growth_long = pd.melt(
    df_pd[['年份', '收入同比增长率(%)', '支出同比增长率(%)', 'GDP同比增长率(%)']],
    id_vars=['年份'],
    value_vars=['收入同比增长率(%)', '支出同比增长率(%)', 'GDP同比增长率(%)'],
    var_name='指标',
    value_name='增长率(%)'
)

chart2 = alt.Chart(df_growth_long).mark_line(
    point=True,
    strokeWidth=2,
    opacity=0.8
).encode(
    x=alt.X('年份:N', axis=alt.Axis(title='年份', labelAngle=0)),
    y=alt.Y('增长率(%):Q', axis=alt.Axis(title='同比增长率(%)')),
    color=alt.Color('指标:N',
                   scale=alt.Scale(
                       domain=['收入同比增长率(%)', '支出同比增长率(%)', 'GDP同比增长率(%)'],
                       range=['#4C78A8', '#F58518', '#2CA02C']
                   ),
                   legend=alt.Legend(title="指标类型")),
    tooltip=['年份', '指标', alt.Tooltip('增长率(%):Q', format='.2f')]
).properties(
    title='浙江省收入、支出、GDP增速对比',
    width=600,
    height=400
)
chart2.show()

alt.Chart(...)

In [37]:
# 3. 财政收入/GDP占比趋势图
chart3 = alt.Chart(df_pd).mark_line(
    point=True,
    color='#9467BD',
    strokeWidth=3
).encode(
    x=alt.X('年份:N', axis=alt.Axis(title='年份', labelAngle=0)),
    y=alt.Y('财政收入/GDP占比(%):Q', 
           axis=alt.Axis(title='财政收入/GDP占比(%)')),
    tooltip=['年份', alt.Tooltip('财政收入/GDP占比(%):Q', format='.2f')]
).properties(
    title='浙江省财政收入/GDP占比趋势（2018-2023年）',
    width=600,
    height=300
)

# 添加合理区间参考线（11-12%）
lower_bound = alt.Chart(pd.DataFrame({'y': [11]})).mark_rule(
    strokeDash=[3, 3],
    color='green',
    opacity=0.5,
    strokeWidth=1
).encode(y='y:Q')

upper_bound = alt.Chart(pd.DataFrame({'y': [12]})).mark_rule(
    strokeDash=[3, 3],
    color='green',
    opacity=0.5,
    strokeWidth=1
).encode(y='y:Q')

# 添加区间标注
annotation_df = pd.DataFrame({
    'text': ['合理区间 11%-12%'],
    'x': ['2020'],
    'y': [11.5]
})

annotation = alt.Chart(annotation_df).mark_text(
    fontSize=10,
    color='green',
    align='center'
).encode(
    x='x:N',
    y='y:Q',
    text='text:N'
)

chart3 = chart3 + lower_bound + upper_bound + annotation
chart3.show()

alt.LayerChart(...)

In [38]:
# 4. 累计增长对比图
df_cumulative_long = pd.melt(
    df_pd[['年份', '收入累计增长率(%)', '支出累计增长率(%)', 'GDP累计增长率(%)']],
    id_vars=['年份'],
    value_vars=['收入累计增长率(%)', '支出累计增长率(%)', 'GDP累计增长率(%)'],
    var_name='指标',
    value_name='累计增长率(%)'
)

chart4 = alt.Chart(df_cumulative_long).mark_area(
    opacity=0.3,
    line=True,
    interpolate='monotone'
).encode(
    x=alt.X('年份:N', axis=alt.Axis(title='年份', labelAngle=0)),
    y=alt.Y('累计增长率(%):Q', axis=alt.Axis(title='累计增长率(%)')),
    color=alt.Color('指标:N',
                   scale=alt.Scale(
                       domain=['收入累计增长率(%)', '支出累计增长率(%)', 'GDP累计增长率(%)'],
                       range=['#4C78A8', '#F58518', '#2CA02C']
                   ),
                   legend=alt.Legend(title="指标类型")),
    tooltip=['年份', '指标', alt.Tooltip('累计增长率(%):Q', format='.2f')]
).properties(
    title='浙江省收入、支出、GDP累计增长对比（以2018年为基期）',
    width=600,
    height=400
)
chart4.show()

alt.Chart(...)

In [39]:
# 5. 复合增长率对比图
cagr_data = pd.DataFrame({
    '指标': ['GDP', '财政收入', '财政支出'],
    '年均复合增长率(%)': [cagr_gdp, cagr_income, cagr_expense],
    '颜色': ['#2CA02C', '#4C78A8', '#F58518']
})

chart5 = alt.Chart(cagr_data).mark_bar(
    width=60
).encode(
    x=alt.X('指标:N', axis=alt.Axis(title='指标'), sort=['GDP', '财政收入', '财政支出']),
    y=alt.Y('年均复合增长率(%):Q', axis=alt.Axis(title='年均复合增长率(%)')),
    color=alt.Color('指标:N',
                   scale=alt.Scale(
                       domain=['GDP', '财政收入', '财政支出'],
                       range=['#2CA02C', '#4C78A8', '#F58518']
                   ))
).properties(
    title=f'浙江省GDP与财政收支年均复合增长率对比（2018-2023）',
    width=500,
    height=350
)

# 添加数值标签
text_labels = chart5.mark_text(
    align='center',
    baseline='bottom',
    dy=-5,
    fontSize=12,
    fontWeight='bold',
    color='black'
).encode(
    text=alt.Text('年均复合增长率(%):Q', format='.2f')
)

chart5 = chart5 + text_labels
chart5.show()

alt.LayerChart(...)

In [40]:



# 6. 财政赤字与GDP关系图
chart6 = alt.Chart(df_pd).mark_circle(
    size=150,
    opacity=0.7
).encode(
    x=alt.X('全省生产总值(亿元):Q', 
           axis=alt.Axis(title='GDP（亿元）')),
    y=alt.Y('财政赤字(亿元):Q', 
           axis=alt.Axis(title='财政赤字（亿元）')),
    color=alt.Color('年份:N', legend=alt.Legend(title="年份")),
    tooltip=['年份', 
             alt.Tooltip('全省生产总值(亿元):Q', format=',.0f'),
             alt.Tooltip('财政赤字(亿元):Q', format=',.0f'),
             alt.Tooltip('财政收入/GDP占比(%):Q', format='.2f')]
).properties(
    title='浙江省财政赤字与GDP关系散点图',
    width=600,
    height=400
)

# 添加趋势线
trend_line = chart6.transform_regression(
    '全省生产总值(亿元)', '财政赤字(亿元)'
).mark_line(
    color='gray',
    strokeDash=[5, 5]
)

chart6 = chart6 + trend_line
chart6.show()


alt.LayerChart(...)